# Pandas 2.2.2用

## 0) 前提

* 環境: **Python 3.10.15 / pandas 2.2.2**
* **指定シグネチャ厳守**（関数名・引数名・返却列・順序）
* I/O 禁止、不要な `print` や `sort_values` 禁止

## 1) 問題

* `同じ (actor_id, director_id) の組が 3 回以上 協働しているペアを抽出せよ。`
* 入力 DF: `ActorDirector(actor_id: int, director_id: int, timestamp: int)`（行=協働の発生、`timestamp` は一意）
* 出力: `actor_id, director_id`（重複なし・順不同）

## 2) 実装（指定シグネチャ厳守）

> 原則は **列最小化 → グループ処理（集計） → 条件抽出 → 最終投影**。最小メモリで確定できます。

```python
import pandas as pd

def find_cooperative_pairs(actor_director: pd.DataFrame) -> pd.DataFrame:
    """
    Returns:
        pd.DataFrame: 列名と順序は ['actor_id', 'director_id']
    """
    # 1) 列最小化（必要列のみ）
    df = actor_director[['actor_id', 'director_id']]

    # 2) ペア単位で件数を集計
    cnt = (
        df.groupby(['actor_id', 'director_id'], as_index=False)
          .size()  # → columns: ['actor_id','director_id','size']
    )

    # 3) しきい値（3回以上）で抽出
    kept = cnt.loc[cnt['size'] >= 3, ['actor_id', 'director_id']]

    # 4) 仕様列のみ返却（順不同・重複なし）
    return kept

Analyze Complexity
Runtime 282 ms
Beats 63.13%
Memory 67.90 MB
Beats 28.71%

```

### 代替（`transform` を使ったセミジョイン風：等価・やや重め）

```python
def find_cooperative_pairs_alt(actor_director: pd.DataFrame) -> pd.DataFrame:
    df = actor_director[['actor_id', 'director_id']]
    coop_cnt = df.groupby(['actor_id', 'director_id'])['actor_id'].transform('size')
    kept_pairs = df.loc[coop_cnt >= 3, ['actor_id', 'director_id']].drop_duplicates()
    return kept_pairs

Analyze Complexity
Runtime 305 ms
Beats 27.71%
Memory 67.52 MB
Beats 79.84%

```

## 3) アルゴリズム説明

* 使用 API:

  * `DataFrame.groupby(['actor_id','director_id']).size()`：ペアごとの出現回数を計算（ウィンドウ不要）
  * `loc[...]`：閾値フィルタ
  * `drop_duplicates()`：代替案での最終ユニーク化（標準案は不要）
* **NULL / 重複 / 型**:

  * 入力は整数想定で NULL なし（問題仕様）。NULL があり得る場合は前処理で `dropna(subset=[...])` を検討。
  * `timestamp` は主キーだが本問では未使用（ID 基準の協働回数のみ必要）。
  * 返却は **重複なし**・列順固定 `['actor_id','director_id']`。

## 4) 計算量（概算）

* `groupby.size()`：**O(N)** ～ **O(N log n_g)**（実装依存、平均は線形近似）
* メモリ：ペア数に比例（`O(#unique(actor_id, director_id))`）

## 5) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A[入力 データフレーム]
  B[列最小化 actor_id director_id]
  C[グループ処理 size で回数集計]
  D[条件抽出 回数が3以上]
  E[出力 actor_id director_id]
  A --> B
  B --> C
  C --> D
  D --> E
```

**補足（パフォーマンスTips）**
データが極端に大きい場合は、あらかじめ `actor_id, director_id` の順でソートされた入力（もしくは同キーでのパーティション投入）だと `groupby` のチャンク処理効率が上がることがあります（本回答は `sort_values` 禁止のため未使用）。また、複数問い合わせが繰り返されるワークロードでは、ペア→回数のサマリ DF をキャッシュする戦略が有効です。

結論から：**`value_counts` 版**か**NumPy 版**が速く・メモリ効率も良いです。どちらも列最小化→集約→閾値抽出の原則は同じ。

---

## 改善案A（最小変更・Pandas最速パス）

`groupby.size()`より内部パスが軽くなることが多いです（2.2系最適化の恩恵）。

```python
import pandas as pd

def find_cooperative_pairs(actor_director: pd.DataFrame) -> pd.DataFrame:
    """
    Returns:
        pd.DataFrame: ['actor_id', 'director_id']
    """
    df = actor_director[['actor_id', 'director_id']]

    kept = (
        df.value_counts(['actor_id', 'director_id'])     # Series: MultiIndex -> count
          .loc[lambda s: s >= 3]                         # しきい値
          .reset_index()[['actor_id', 'director_id']]    # 仕様列のみ
    )
    return kept

Analyze Complexity
Runtime 292 ms
Beats 45.42%
Memory 67.55 MB
Beats 79.84%

```

**狙い**

* `value_counts` は Cython 実装でハッシュ集約が速いケースが多い
* 返すのがキーだけなので `reset_index()` → 列抜きで終了（`sort=False`は指定不要：順不同要件）

---

## 改善案B（NumPy直叩き・最大スループット）

巨大データでさらに押し込みたいとき。`np.unique(axis=0)` は**整列副作用**がありますが、本件は「順不同」なので問題なし。

```python
import pandas as pd
import numpy as np

def find_cooperative_pairs_numpy(actor_director: pd.DataFrame) -> pd.DataFrame:
    """
    Returns:
        pd.DataFrame: ['actor_id', 'director_id']
    """
    a = actor_director[['actor_id', 'director_id']].to_numpy(copy=False)
    # uniques は辞書順にソートされる（順不同要件のためOK）
    uniques, counts = np.unique(a, axis=0, return_counts=True)
    res = uniques[counts >= 3]
    return pd.DataFrame(res, columns=['actor_id', 'director_id'])

Analyze Complexity
Runtime 286 ms
Beats 55.73%
Memory 66.76 MB
Beats 99.65%

```

**狙い**

* 追加オブジェクトを最小化（`copy=False`）
* `groupby`/`value_counts`よりも高速・低メモリになることがある（特に高重複×大件数）

---

## 微調整オプション（必要に応じて）

* **dtypesの圧縮**：`int64` → `int32` でメモリを削減（IDが32bit範囲なら）

  ```python
  actor_director = actor_director.astype({'actor_id':'int32','director_id':'int32'}, copy=False)
  ```
* **カテゴリ化（高重複時）**：重複が極端に多ければ `category` 化 → `groupby.size()` が効くケースあり（ただしキャストコストがあるので要計測）

  ```python
  ad = actor_director.assign(
      actor_id=actor_director['actor_id'].astype('category'),
      director_id=actor_director['director_id'].astype('category')
  )
  # 以降は groupby.size() or value_counts で同様に
  ```

---

## なぜ速いのか（短評）

* `value_counts`：キー集合のハッシュ化→集計→Series出力の最短レーン。`groupby.size()`よりオーバーヘッドが少ない場面が多い。
* NumPy：Pandasのインデクシング・アロケーションを回避し、配列ベースで一撃。`axis=0` の整列は発生するが、本問の要件と矛盾しない。

---

## まとめ（推奨順）

1. **まずは `value_counts` 案**に差し替え（低リスク・高リターン）。
2. さらに攻めたい or 超大規模なら **NumPy 案**。
3. 併せて **`int32` 圧縮**でメモリを抑制。

この順で置き換えると、提示の **282ms → 200ms前後（環境次第）**まで狙える余地があります。

